In [162]:
import numpy as np
import pandas as pd
from scipy.stats import iqr
from sklearn.neural_network import MLPClassifier

In [163]:
r_x = 3
r_e = 1
r_xe = r_x+r_e
r_z = (10,5,3)
n = 1000

choose = lambda x,y: int(np.product(np.arange(x,x-y,-1))/np.product(np.arange(1,y+1)))
bind = lambda *x: np.hstack(x) 
relu = lambda x: np.maximum(x,0)
softmax = lambda x: np.divide(np.exp(x),np.exp(x).sum(1).reshape(-1,1))

ones = np.ones((n,1))
w = np.ones((1,2))
for i in range(2,r_xe):
    w = bind(bind(np.ones((i,1)),np.eye(i)).T,\
             bind(np.zeros((w.shape[0],1)),w).T).T
xe = np.random.normal(size=(n,r_xe))
z = np.random.normal(size=(n,choose(r_xe,2)))
for i in range(w.shape[0]):
    if w[i,-r_e:].sum() != 0 and w[i,:r_x].sum() != 0:
        w[i,:] = 0
xe = xe + z@w

In [164]:
wmat = []
layers = []
last = xe
for i in range(len(r_z)):
    w_tmp = np.random.uniform(size=(last.shape[1]+1,r_z[i]))*2-1
    last = relu(bind(ones,last)@w_tmp)
    wmat += [w_tmp]
    layers += [last]
[i.shape for i in layers]
y = softmax(last).argmax(1)
x = xe[:,:r_x]
np.unique(y,return_counts=True)

(array([0, 1], dtype=int64), array([482, 518], dtype=int64))

In [165]:
def mypred(x):
    last = x
    ones = np.ones((x.shape[0],1))
    for i in range(len(r_z)):
        w_tmp = wmat[i]
        last = relu(bind(ones,last)@w_tmp)
    return softmax(last)
mypred(bind(x,np.zeros((n,1))))

array([[0.74255243, 0.13696309, 0.12048448],
       [0.53076311, 0.29324995, 0.17598693],
       [0.7283656 , 0.1358172 , 0.1358172 ],
       ...,
       [0.3742735 , 0.36550817, 0.26021833],
       [0.84493115, 0.07753442, 0.07753442],
       [0.27953082, 0.44093836, 0.27953082]])

In [166]:
dx = iqr(x,axis=0)/100

In [167]:
dx

array([0.02500419, 0.02442524, 0.02434061])

In [168]:
bind(x,np.zeros((n,1)))

array([[-1.81385159, -1.30198318, -2.78250639,  0.        ],
       [-2.25077839, -0.21811438, -1.47549428,  0.        ],
       [ 1.25460607, -1.21195789, -4.04941126,  0.        ],
       ...,
       [ 0.11561989,  0.47961459, -0.33976203,  0.        ],
       [-0.31112215, -0.90165573, -3.53742455,  0.        ],
       [ 3.01068905,  0.15500319, -2.13107257,  0.        ]])

In [169]:
x = xe[:,:r_x]
dydx = []
for i in range(r_x):
    x_tmp = x.copy()
    x_tmp[:,i] += dx[i]
    dy = mypred(bind(x_tmp,np.zeros((n,1))))-mypred(bind(x,np.zeros((n,1))))
    dydx += [(dy/dx[i]).mean(0)]
dydx = np.array(dydx)
dydx

array([[-0.01344369,  0.00099491,  0.01244877],
       [-0.02749536,  0.01638431,  0.01111106],
       [-0.06158452,  0.04635229,  0.01523222]])

In [170]:
(x_tmp[:,2]+dx[2])

array([-2.73382517e+00, -1.42681306e+00, -4.00073005e+00,  4.33780661e-01,
        2.20103631e+00, -1.11018424e+00, -2.59079111e+00, -8.35283897e-01,
        4.65849906e+00,  1.97602536e+00,  2.06978732e+00,  4.18859127e+00,
        6.37442584e-01,  1.14754935e+00,  2.36861994e+00, -1.11543113e+00,
       -2.19124317e+00, -1.96655893e+00,  4.31954582e+00,  1.70516594e+00,
        2.20759222e-01,  5.52547691e-01,  7.27407129e-01, -1.83692293e+00,
       -1.20146167e+00, -2.36627813e+00,  2.48964827e+00,  1.70893504e+00,
        1.55896842e+00,  3.22321125e+00,  2.37092841e+00,  2.24729286e+00,
        1.75023526e+00, -2.29668426e+00,  6.97396854e-01,  1.62458629e+00,
       -9.29521717e-01,  5.38094445e+00,  1.21129370e-01, -1.06914096e+00,
        3.94577912e-01,  1.98132996e-01, -1.45087371e+00,  1.24456496e+00,
       -3.53875979e-01, -1.56931917e+00,  8.54486305e-01, -2.33546062e+00,
        3.32153987e+00, -1.88858256e+00, -4.34259062e-01,  7.73481914e-01,
       -3.73932716e+00, -

In [171]:
y.shape

(1000,)

In [181]:

mlp = MLPClassifier(hidden_layer_sizes=(5),max_iter=10000,alpha=1)
model = mlp.fit(x,y)
accur = (y==model.predict(x)).astype(int).sum()/n
accur

0.764

In [140]:
edydx = []
for i in range(r_x):
    x_tmp = x.copy()
    x_tmp[:,i] += dx[i]
    dy = mlp.predict(x_tmp)-mlp.predict(x)
    edydx += [(dy/dx[i]).mean(0)]
edydx = np.array(dydx)
edydx

array([[-0.01003079, -0.0726964 ,  0.08272719],
       [ 0.00464443, -0.00025814, -0.00438629],
       [-0.03467526, -0.03304805,  0.06772331]])

In [141]:
((edydx-dydx)**2).mean()

0.0

In [150]:
iqr(x,0)

array([2.35632053, 2.24911245, 2.27271071])

In [153]:
?iqr

In [154]:
iqr(np.array(x),axis=0)

array([2.35632053, 2.24911245, 2.27271071])